# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [2]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [15]:
# 1) Seu código aqui
url = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'

In [16]:
r = requests.get(url)

In [17]:
r.status_code

200

In [18]:
r.json()

{'items': [{'id_pleito': 13271,
   'tipo_interessado': 'Estado',
   'interessado': 'Rio de Janeiro',
   'cod_ibge': 33,
   'uf': 'RJ',
   'num_pvl': None,
   'status': 'Encaminhado à PGFN com manifestação técnica favorável',
   'num_processo': '17944.001999/2008-12',
   'data_protocolo': '2009-07-01T00:00:00Z',
   'tipo_operacao': 'Operação contratual externa (com garantia da União)',
   'finalidade': 'Desenvolvimento rural',
   'tipo_credor': 'Instituição Financeira Internacional',
   'credor': 'Banco Internacional para Reconstrução e Desenvolvimento',
   'moeda': 'Dólar dos EUA',
   'valor': 39500000,
   'pvl_assoc_divida': 1,
   'pvl_contradado_credor': 0,
   'data_status': '07/07/2009'},
  {'id_pleito': 5923,
   'tipo_interessado': 'Estado',
   'interessado': 'Rio de Janeiro',
   'cod_ibge': 33,
   'uf': 'RJ',
   'num_pvl': None,
   'status': 'Encaminhado à PGFN com manifestação técnica favorável',
   'num_processo': '17944.000234/2012-41',
   'data_protocolo': '2012-04-16T00:00:00

In [20]:
data_json = r.json()

In [26]:
# 2) Seu código aqui
df = pd.DataFrame(data_json['items']) # transforma em dataframe e salva na variável df

In [27]:
df['status'].value_counts().head(3)

status
Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Name: count, dtype: int64

In [28]:
# 3) Seu código aqui
df['ano_status'] = df['data_status'].str.slice(0, 4)

In [29]:
# 4) Seu código aqui
df['ano_status'].value_counts().sort_index()

ano_status
01/0    4
02/0    1
03/0    1
04/0    5
04/1    3
05/0    1
06/0    5
07/0    2
09/1    1
10/0    2
11/0    1
13/0    1
14/0    2
16/0    1
18/0    2
19/0    1
20/0    1
20/1    1
21/1    1
22/0    1
24/0    1
25/1    2
26/0    1
27/0    4
27/1    2
28/0    3
29/0    2
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [31]:
#1) Seu código aqui
def consultar_pvl(uf, tipo_interessado):
    url = f"http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}"
    r = requests.get(url)
    if r.status_code == 200:
        dados = r.json()
        df = pd.DataFrame(dados['items'])
        return df
    else:
        print("Erro ao acessar API:", r.status_code)
        return pd.DataFrame()  # Retorna vazio se erro

In [32]:
# 2) Seu código aqui
df_mg_estado = consultar_pvl('MG', 'Estado')
qtd_arquivadas = df_mg_estado[df_mg_estado['status'] == 'Arquivado por decurso de prazo'].shape[0]
print(f"Total: {qtd_arquivadas}")

Total: 1


In [33]:
# 3) Seu código aqui
df_ba_municipio = consultar_pvl('BA', 'Município')
deferidos = df_ba_municipio[df_ba_municipio['status'].str.contains('Deferido', case=False, na=False)]
mais_solicitacoes = deferidos['interessado'].value_counts().idxmax()
print(f"Município com mais solicitações deferidas: {mais_solicitacoes}")

Município com mais solicitações deferidas: Luís Eduardo Magalhães


In [38]:
# 4) Seu código aqui
df_ba_estado = consultar_pvl('BA', 'Estado')
df_ba_estado.to_csv('solicitacoes_bahia_estado.csv', index=False, sep=',')
print("Arquivo salvo com sucesso.")

Arquivo salvo com sucesso.
